# Training Notebook

All our training is done using 10k / 2k / 2k sized data for train / val / test.

This notebook narrows images to only those with sports in them. We then randomly sample these to get 10k / 2k / 2k. This gives us a slightly smaller vocab, but same size dataset. 

We only train on images containing sports categories. 

This is an alternative version of sport_v2. It uses different encoder architecture: one final fully connected layer rather than the two layers we use everywhere else. NOTE: WE IMPLEMENTED THIS OUTSIDE OUR USUAL WORK FLOW THAT THE CODE WAS DESIGNED FOR. IT INVOLVED TEMPORARILY CHANGING THE CODE IN THE ENCODER CLASS, AND THIS NOTEBOOK WILL NOT SUCCESSFULLY RUN WITHOUT CHANGING THE ENCODER MODULE.

We leave this notebook in for completeness, and because it is a pointer for future code development. We would definitely have the option for a different encoder architecture in future.

## Import libraries

In [1]:
from get_loader import get_loader
from models import Encoder, Decoder
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from data_prep_utils import *
from pathlib import Path
import json

## Load train and validation loaders

In [2]:
#image_path = '../../CW/Data/train2017'
#captions_path = '../../CW/Data/annotations_trainval2017/annotations/captions_train2017.json'
IMAGE_PATH = '../Datasets/coco/images/train2017'
CAPTIONS_PATH = '../Datasets/coco/annotations/' #captions_train2017.json'
FREQ_THRESHOLD = 4
CAPS_PER_IMAGE = 5
BATCH_SIZE = 128
SHUFFLE = True

# root of the name to save or load captions files
CAPTIONS_NAME = 'sports_encoder_v2'
SUPER_CATEGORIES = ['sports'] # should be list of eligible coco super categories, or None to include all images

# for encoder and decoder
EMBED_SIZE = 512  # dimension of vocab embedding vector
HIDDEN_SIZE = 512
NUM_LAYERS = 3  # hidden layers in LTSM

# training parameters
PRINT_EVERY = 100
TOTAL_EPOCH = 20
CHECKPOINT = '../model/model_sport_encoder_v2' # there is no v1 for sports:
# v2 is consistent with previous tests as v2 parameters are shared across data sets

In [3]:
# of images or reduce the size of the data
# this will write files to 'Datasets/coco/annotations' as 
#     [save_name]_captions_train.json
#     [save_name]_captions_val.json
#     [save_name]_captions_test.json

prepare_datasets(train_percent = 0.87, super_categories=['sports'],
                 max_train=15000, max_val=2000, max_test=2000,
                 save_name=CAPTIONS_NAME, random_seed=42)

# we explicitly build the vocab here. We use frequency threshold, and we build
# vocab from the specified captions file: we're using the training data
# we save the vocab to a name consistent with our training captions data so that 
# we can load a vocab consistent with the specific training run we've used.
build_vocab(freq_threshold = FREQ_THRESHOLD, 
            captions_file=f'{CAPTIONS_NAME}_captions_train.json',
            vocab_save_name=CAPTIONS_NAME)

train dataset has 15000 images
 val dataset has 2000 images
 test dataset has 938 images
There are 75039 captions in the data set
With FREQ_THRESHOLD = 4, vocab size is 2921


In [4]:
with open(f'../vocabulary/{CAPTIONS_NAME}word2idx.json', 'r') as f:
    word2idx = json.load(f)
vocab_size = len(word2idx)

In [5]:
train_loader_params = {
    'images_path': IMAGE_PATH,
    'captions_path': CAPTIONS_PATH + f'{CAPTIONS_NAME}_captions_train.json',
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': 5,
    'batch_size': BATCH_SIZE,
    'shuffle': SHUFFLE,
    'mode': 'train',
    # 'idx2word': None,
    'word2idx': word2idx
}

train_loader, train_dataset = get_loader(**train_loader_params)

val_loader_params = {
    'images_path': IMAGE_PATH,
    'captions_path': CAPTIONS_PATH + f'{CAPTIONS_NAME}_captions_val.json',
    'freq_threshold': FREQ_THRESHOLD,
    'caps_per_image': 3,
    'batch_size': BATCH_SIZE,
    'shuffle': SHUFFLE,
    'mode': 'validation',
    # 'idx2word': train_dataset.vocab.idx2word,
    'word2idx': word2idx
}

val_loader, val_dataset = get_loader(**val_loader_params)

print(f"Length of training dataloader: {len(train_loader)}, Length of testing dataloader: {len(val_loader)}")
print(f"Length of vocabulary: {len(train_dataset.vocab.idx2word)}")

Length of training dataloader: 586, Length of testing dataloader: 47
Length of vocabulary: 2921


## Load the model

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"We are using {device}.")

We are using cuda.


In [7]:
encoder = Encoder(embed_size=EMBED_SIZE, pretrained=True)
decoder = Decoder(embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, vocab_size=vocab_size, num_layers=NUM_LAYERS)

In [8]:
# the loss is a cross entropy loss and ignore the index of <PAD> since it doesn't make any difference
criterion = nn.CrossEntropyLoss(ignore_index=train_dataset.vocab.word2idx["<PAD>"]).cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss(ignore_index=train_dataset.vocab.word2idx["<PAD>"])

# combine the parameters of decoder and encoder
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# Adam optimizer
opt_pars = {'lr':1e-3, 'weight_decay':1e-3, 'betas':(0.9, 0.999), 'eps':1e-08}
optimizer = optim.Adam(params, **opt_pars)

In [9]:
model_params = {
    'save_path': CHECKPOINT,
    'batch_size': BATCH_SIZE,
    'embed_size': EMBED_SIZE,
    'hidden_size': HIDDEN_SIZE,
    'num_layers': NUM_LAYERS,
    'vocab_size': len(train_dataset.vocab.idx2word)
}

save_params(**model_params)

## Training

In [10]:
train_params = {
    'encoder': encoder,
    'decoder': decoder,
    'criterion': criterion,
    'optimizer': optimizer,
    'train_loader': train_loader,
    'val_loader': val_loader,
    'total_epoch': TOTAL_EPOCH,
    'device': device,
    'checkpoint_path': CHECKPOINT,
    'print_every': PRINT_EVERY,
    'load_checkpoint': False
}

training_loss, validation_loss = train(**train_params) 

Epoch: [0/20]          || Step: [0/586]         || Average Training Loss: 7.9808
Epoch: [0/20]          || Step: [100/586]       || Average Training Loss: 4.6321
Epoch: [0/20]          || Step: [200/586]       || Average Training Loss: 4.1362
Epoch: [0/20]          || Step: [300/586]       || Average Training Loss: 3.8262
Epoch: [0/20]          || Step: [400/586]       || Average Training Loss: 3.6163
Epoch: [0/20]          || Step: [500/586]       || Average Training Loss: 3.4604
Epoch: [0/20]          || Step: [0/47]          || Average Validation Loss: 2.9689
****************************************************************************************************
Epoch: [0/20] || Training Loss = 3.35 || Validation Loss: 2.63 || Time: 23.740939
****************************************************************************************************
Epoch: [1/20]          || Step: [0/586]         || Average Training Loss: 2.7639
Epoch: [1/20]          || Step: [100/586]       || Average Trainin